In [1]:
## ZeroShot learning for SIB-200 dataset by generating text

from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast
import os
import pandas as pd
import torch
import warnings
from transformers import XGLMTokenizer, XGLMForCausalLM

model_address = "bigscience/bloom-1b1"

# Filter out FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

# Assuming data directory contains multiple subdirectories with test.tsv files
data_directory = "sib-200/data/annotated"

if model_address.startswith("facebook"):
    # Load ZeroShot learning model and tokenizer
    model = XGLMForCausalLM.from_pretrained(model_address)
    tokenizer = XGLMTokenizer.from_pretrained(model_address, padding_side='left')
if model_address.startswith("bigscience"):
    # Load ZeroShot learning model and tokenizer
    model = BloomForCausalLM.from_pretrained(model_address)
    tokenizer = BloomTokenizerFast.from_pretrained(model_address)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to translate given texts to English
def translate_batch(texts, input_language, output_language="English", batch_size=5):
    with torch.no_grad():  # Disable gradient calculation
        generated_texts = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            prompts = [f"{input_language}: {text} \n{output_language}:" for text in batch_texts]
            inputs = tokenizer(prompts, padding=True, return_tensors="pt").to(device)
            tokens = tokenizer.batch_encode_plus(prompts, add_special_tokens=True, padding=True, return_tensors="pt")['input_ids']
            result_length = tokens.shape[1] + 100
            generated_batch = model.generate(inputs["input_ids"], max_length=result_length)
            for generated_text in generated_batch:
                generated_texts.append(tokenizer.decode(generated_text, skip_special_tokens=True))
            del inputs
            torch.cuda.empty_cache()
    return generated_texts

# Output directory for saving DataFrames
output_directory = "MT-task/sib-200/" + model_address[model_address.find('/')+1:] + "/"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

language_df = pd.read_excel("SIB-200 languages - ACL.xlsx")

# Iterate through rows and compare predicted category with actual category
for index, row in language_df.iterrows():
    language = row['Language Name']
    folder = row['Folder Name']
    print(language)
    if language == "English":
        continue
    if f"{folder}.csv" in os.listdir(output_directory):
        continue
    # Initialize an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['text', 'translated_text'])

    subdir = os.path.join(data_directory, folder)
    for file in os.listdir(subdir):
        # Check if the file is a test.tsv file
        if file.endswith("test.tsv"):
            file_path = os.path.join(subdir, file)
            
            # Check if the file is already present
            results_file_path = os.path.join(output_directory, f'{folder}.csv')
            if os.path.exists(results_file_path):
                print(f"Output file {results_file_path} already exists. Skipping...")
                continue

            # Read the test.tsv file into a DataFrame
            df = pd.read_csv(file_path, sep='\t')

            # Get all texts from the DataFrame
            texts = df['text'].tolist()

            # Predict translations using your ZeroShot learning model in batches
            generated_texts = translate_batch(texts=texts, input_language=language)

            # Append the results to the DataFrame
            results_df['text'] = texts
            results_df['translated_text'] = generated_texts

            # Save the results DataFrame to a CSV file in the output directory
            results_file_path = os.path.join(output_directory, f'{folder}.csv')
            results_df.to_csv(results_file_path, index=False)

            print(f"Results saved to {results_file_path}")

Aceh
Aceh
Mesopotamian Spoken Arabic
Arabic, Ta’izzi-Adeni Spoken
Tunisian Spoken Arabic
Afrikaans
South Levantine Arabic
Akan
Tosk Albanian
Amharic
Levantine Arabic
Standard Arabic
Standard Arabic
Najdi Spoken Arabic
Moroccan Spoken Arabic
Egyptian Spoken Arabic
Assamese
Asturianu
Awadhi
Central Aymara
South Azerbaijani
North Azerbaijani
Bashkort
Bamanankan
Bali
Belarusian
Bemba
Bengali
Bhojpuri
Banjar
Banjar
Central Tibetan
Bosnian
Bugis
Bulgarian
Catalan
Cebuano
Czech
Chokwe
Central Kurdish
Crimean Tatar
Welsh
Danish
Standard German
Southwestern Dinka
Jula
Dzongkha
Greek
English
Esperanto
Estonian
Euskera
Éwé
Faroese
Fijian
Finnish
Fon
French
Friulian
Nigerian Fulfulde
West Central Oromo
Scottish Gaelic
Irish
Galician
Guarani
Gujarati
Haitian Creole
Hausa
Hebrew
Hindi
Chhattisgarhi
Croatian
Hungarian
Armenian
Igbo
Ilocano
Indonesian
Icelandic
Italian
Javanese
Japanese
Kabyle
Jingpho
Kamba
Kannada
Kashmiri
Kashmiri
Georgian
Kazakh
Kabiyè
Kabuverdianu
Halh Mongolian
Khmer
Gikuyu
Kinya